In [6]:
import sys, os
sys.path.append(os.path.realpath(".."))

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
reload(cta)

<module 'clinical_text_analysis' from '/home/ms994/secondDbmi/clinical_text_analysis.py'>

In [2]:
import ensembleReader as er

In [92]:
reload(er)
dataReader = er.EdfDatasetEnsembler("dev_test", "01_tcp_ar",)

In [93]:
reload(read)
slr = read.SeizureLabelReader(sampleInfo=dataReader.sampleInfo)

In [88]:
seizInfo = slr[0]

In [94]:
total = slr[:]

Starting 4 processes
retrieving: 0
retrieving: 100
retrieving: 200
retrieving: 300
retrieving: 400
retrieving: 500
retrieving: 600
retrieving: 700
retrieving: 800
retrieving: 900
retrieving: 1000
retrieving: 1100
retrieving: 1200
retrieving: 1300
retrieving: 1400
retrieving: 1500
retrieving: 1600
retrieving: 1700
retrieving: 1800
retrieving: 1900
retrieving: 2000
retrieving: 2100
retrieving: 2200
retrieving: 2300
retrieving: 2400
retrieving: 2500
retrieving: 2600
retrieving: 2700
retrieving: 2800
retrieving: 2900
retrieving: 3000
retrieving: 3100
retrieving: 3200
retrieving: 3300
retrieving: 3400
retrieving: 3500
retrieving: 3600
retrieving: 3700
retrieving: 3800
retrieving: 3900
retrieving: 4000
retrieving: 4100
retrieving: 4200
retrieving: 4300
retrieving: 4400
retrieving: 4500
retrieving: 4600
retrieving: 4700
retrieving: 4800
retrieving: 4900
retrieving: 5000
retrieving: 5100
retrieving: 5200
retrieving: 5300
retrieving: 5400
retrieving: 5500
retrieving: 5600
retrieving: 5700
retri

In [96]:
sum(total)/len(total)

0.0686496267812712

In [ ]:
total.sum

In [25]:
dataReader.sampleInfo[0]

{'token_file_path': '/home/ms994/v1.5.0/edf/dev_test/01_tcp_ar/069/00006986/s001_2010_09_30/00006986_s001_t001.edf',
 'sample_num': 267,
 'within_token_num': 0,
 'sample_width': Timedelta('0 days 00:00:04')}

In [27]:
res = dataReader.sampleInfo[0].sample_width * dataReader.sampleInfo[0].sample_num

In [34]:
start = pd.Timedelta(res).seconds

In [35]:
end = pd.Timedelta(res + dataReader.sampleInfo[0].sample_width).seconds

In [63]:
start, end

(1068, 1072)

In [45]:
seizInfo

,start,end,label,p
0,0,1,bckg,1
1,1,62.28,cpsz,1
2,62.28,1196,bckg,1


In [66]:
seizInfo.start > start

0    False
1    False
2    False
Name: start, dtype: bool

In [72]:
seizSlice = seizInfo.loc[(seizInfo.end > start) & ((seizInfo.end <= end)).shift(1)]

In [76]:
(seizSlice.label != "bckg").any()

False

In [17]:
dataReader[len(dataReader) - 1]

array([[ -8.20419609,   1.67389091,  -5.08097727, ...,  -6.46100384,
         -1.15879583,  -7.11470127],
       [ -9.88747804,   6.41976591,  -8.43739631, ...,  -4.98027789,
          0.53376398,  -2.58960629],
       [ -8.29999841,   6.97976468, -11.63903722, ...,  -0.54031085,
         -0.85139844,  -5.96522476],
       ...,
       [ 15.19887989,  -0.93114334,  10.4655374 , ...,   9.99249828,
         29.96026345, -10.51160814],
       [ 16.99928043,   2.44134432,  10.78463447, ...,  10.63653747,
         31.29970288,  -9.44407936],
       [ 18.0702209 ,   5.99205425,  18.85250359, ...,  12.6842008 ,
         34.04422281,  -1.23419783]])

In [26]:
genderFiles = cta.demux_to_tokens(cta.getGenderAndFileNames("dev_test", "01_tcp_ar", convert_gender_to_num=True))

In [4]:
samplingInfo = pkl.load(open("../test_standardized_edf_ensemble_sample_info.pkl","rb"))

In [5]:
reload(er)
ale = er.AdditionalLabelEndpoints("combined", "01_tcp_ar", sampleInfo=samplingInfo)

In [6]:
gends = ale.get_genders()

KeyError: '/home/ms994/v1.5.0/edf/combined/01_tcp_ar/000/00007475/s003_2013_03_22/00007475_s003_t000.edf'

In [ ]:
agesDictItems = cta.demux_to_tokens(cta.getAgesAndFileNames(self.split, self.ref))
agesDict = {}
for index in range(len(agesDictItems)):
    key = agesDictItems[0][index]
    val = agesDictItems[1][index]
    agesDict[key] = val

In [3]:
import keras
import pickle as pkl

Using TensorFlow backend.


In [4]:
testData = pkl.load(open("../standardized_combined_simple_ensemble_test_data.pkl", 'rb'))
def generate_x_y(data):
    x_data = np.stack([datum[0] for datum in data])
    x_data = x_data.reshape((*x_data.shape, 1))
    x_data.transpose(0, 2, 1, 3)
    y_data = np.array([datum[1] for datum in data])
    y_data = keras.utils.to_categorical(y_data)
    return x_data, y_data
testDataX, testDataY = generate_x_y(testData)
del testData

In [5]:
model = keras.models.load_model("../ZIERFTVVDCSJGPLD.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
from keras.utils import multi_gpu_model

In [7]:
model = multi_gpu_model(model, 2)

In [ ]:
y_pred = model.predict(testDataX.transpose(0, 2, 1, 3))

In [11]:
from sklearn.metrics import roc_auc_score

In [12]:
roc_auc_score(testDataY.argmax(axis=1), y_pred.argmax(axis=1))

0.5630877449014666

In [29]:
log_pred = np.log(y_pred + 1)

In [14]:
samplingInfo = pkl.load(open("../test_standardized_edf_ensemble_sample_info.pkl","rb"))

In [15]:
reload(read)

<module 'data_reader' from '/home/ms994/dbmi_eeg_clustering/data_reader.py'>

In [16]:
edf_tokens = list(set([samplingInfo[key]["token_file_path"] for key in samplingInfo.keys()]))

In [17]:
import ensembleReader as er
ensembler = er.EdfDatasetEnsembler("combined", "01_tcp_ar", generate_sample_info=False, edf_tokens=edf_tokens)

In [ ]:
len(edf_tokens)

In [18]:
ensembler.sampleInfo = samplingInfo
ensembler.labels = testDataY.argmax(axis=1)
ensembler.edf_tokens = edf_tokens

In [ ]:
from addict import Dict

In [32]:
trueLabel, pred = ensembler.getEnsemblePrediction(log_pred, mode=er.EdfDatasetEnsembler.ENSEMBLE_PREDICTION_EQUAL_VOTE)

In [38]:
y_pred.sum(0) / y_pred.sum()

array([0.2667956 , 0.73320466], dtype=float32)

In [37]:
testDataY.sum(0) / (testDataY.sum())

array([0.38224864, 0.61775136], dtype=float32)

In [33]:
roc_auc_score(trueLabel, pred)

0.5993215389900377

In [45]:
reload(er)
ale = er.AdditionalLabelEndpoints("combined", "01_tcp_ar", ensembler=ensembler)

In [46]:
ale.get_ages()

KeyError: '/home/ms994/v1.5.0/edf/combined/01_tcp_ar/000/00007475/s003_2013_03_22/00007475_s003_t000.edf'